<a href="https://colab.research.google.com/github/rezoo/movis/blob/colab/Quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movis: Video Editing as a Code

This Jupyter notebook goes through the basic usage of Movis.

- Install Movis
- Write a simple python script that generates a video with a background and a text
- Write a bit practical python script that concatenates a title logo and a video

### Install movis

In [1]:
pip install git+https://github.com/rezoo/movis.git

In [2]:
import os

import movis as mv
from PIL import Image

# It has been found that errors occur when using PySide only on Google colab.
# To avoid this, we explicitly set an environment variable.
os.environ['QT_QPA_PLATFORM'] = 'offscreen'

Now let's make a simple video.

In [3]:
w, h = 960, 540

title = mv.layer.Composition(size=(w, h), duration=4.0)

title.add_layer(mv.layer.Rectangle(title.size, color='seagreen'), 'bg')
title.add_layer(
    mv.layer.Text('Big Buck Bunny', font_size=60.0, font_family='Liberation Sans', color='#ffffff'),
    name='title',
    position=(w / 2, h / 2))
title.add_layer(
    mv.layer.Text('By Blender Foundation', font_size=20.0, font_family='Liberation Sans', color='#ffffff'),
    name='subtitle',
    position=(w / 2, h / 2 + 50))

By converting an ``ndarray`` to a ``PIL.Image``, you can view the image directly in the Notebook. In this case, we are outputting a still image at the ``0.0`` second.

In [4]:
Image.fromarray(title(0.0))

Let's try adding an animation. The target layer includes common attributes such as position, scale, and opacity. By adding keyframes to these attributes, you can animate them.

In [5]:
# Easings define the easing between keyframes. "ease_out{N}" refers to an ease-out animation.
# The larger the value of N, the more abrupt the animation transition will be.
title['title'].scale.enable_motion().extend(
    keyframes=[0.0, 1.5],
    values=[0.0, 1.0],
    easings=['ease_out8'],
)
title['title'].opacity.enable_motion().extend(
    keyframes=[0.0, 1.5],
    values=[0.0, 1.0],
)

p = title['subtitle'].position.init_value
title['subtitle'].position.enable_motion().extend(
    keyframes=[0.5, 1.5],
    values=[(p[0], p[1] + 10), p],
    easings=['ease_out8']
)
title['subtitle'].opacity.enable_motion().extend(
    keyframes=[0.5, 1.5],
    values=[0.0, 1.0]
)

For a preview on IPython Notebook, use `render_and_play()`. The `preview_level` defines the resolution of the output video. For instance, if `preview_level=2`, the resolution will be halved.

In [6]:
title.render_and_play(preview_level=1)

It does not look bad.

Next, download the video itself. Let's use the video provided by Blender foundation.

In [7]:
!curl -O https://commondatastorage.googleapis.com/gtv-videos-bucket/sample/BigBuckBunny.mp4

In [8]:
video = mv.layer.Video('./BigBuckBunny.mp4')

For example, if you want to see a picture of the video layer after 10 seconds, you can simply use the following:

In [9]:
frame = video(10.0)
Image.fromarray(frame)

In this case, the resolution of the video is relatively small, so it must be scaled down by a reasonable value.

In [10]:
scale = w / frame.shape[1]

scene = mv.layer.Composition(size=(w, h), duration=15.0)
scene.add_layer(
    video, name='content', scale=scale)

In [11]:
Image.fromarray(scene(10.0))

It appears to have been scaled down correctly.

Next, composite the title and the body of the video. For simplicify, in this case we will create a video that is only 15 seconds long.

In [12]:
scene = mv.layer.Composition(size=(w, h), duration=15.0)
scene.add_layer(video, name='content', scale=scale, offset=4.0)
scene.add_layer(title, name='title')

p = scene['title'].position.init_value
scene['title'].position.enable_motion().extend(
    keyframes=[3.0, 4.0], values=[p, (p[0] + w, p[1])], easings=['ease_in8']
)
scene.render_and_play(preview_level=2)

Looks good. Finally, we output these videos using `scene.write_video()`.

Note here that we explicitly specify `audio=True` to include audio in the videos (movis does not output audio by default)

In [13]:
scene.write_video('output.mp4', audio=True)

Movis can create more sophisticated animations in videos by using a variety of compositions and effects. For more details, please refer to the [Examples on GitHub](https://github.com/rezoo/movis/tree/main/examples).